In [1]:
import cv2
import mediapipe as mp
import time
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import time

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
poses = mp_pose.Pose(
    static_image_mode=False,  #静止画かどうか
    # UPPER_BODY_ONLY=False,   #上半身のみかどうか
    # SMOOTH_LANDMARKS=True,   #ジッターを減らすかどうか
    min_detection_confidence=0.7, #検出成功の最低信頼値
    min_tracking_confidence=0.7 #追跡成功の最小信頼値
     ) 

cap = cv2.VideoCapture(1)
vid = cv2.VideoCapture("mv/umap.mp4")
# 59.94fps 1920p1080

In [3]:
print(vid.get(cv2.CAP_PROP_FPS))

30.0


In [ ]:
# 座標をnumpyにする
def landmark2np(pose_landmarks):
    li = []
    for j in (pose_landmarks.landmark):
        li.append([j.x, j.y, j.z])

    return np.array(li) - li[0]

# コサイン類似度を計算する
def manual_cos(A, B):
    dot = np.sum(A*B, axis=-1)
    A_norm = np.linalg.norm(A, axis=-1)
    B_norm = np.linalg.norm(B, axis=-1)
    cos = dot / (A_norm*B_norm+1e-7)
    print(cos[1:].mean())

    return cos[1:].mean()

# 動画を読み込み、FPSを変更して別名で保存する関数
def m_speed_change(path_in, path_out, scale_factor, color_flag):
    # 動画読み込みの設定
    movie = cv2.VideoCapture(path_in)
 
    # 動画ファイル保存用の設定
    fps = int(movie.get(cv2.CAP_PROP_FPS))                                  # 元動画のFPSを取得
    fps_new = int(fps * scale_factor)                                       # 動画保存時のFPSはスケールファクターをかける
    w = int(movie.get(cv2.CAP_PROP_FRAME_WIDTH))                            # 動画の横幅を取得
    h = int(movie.get(cv2.CAP_PROP_FRAME_HEIGHT))                           # 動画の縦幅を取得
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')                     # 動画保存時のfourcc設定（mp4用）
    video = cv2.VideoWriter(path_out, fourcc, fps_new, (w, h), color_flag)  # 動画の仕様（ファイル名、fourcc, FPS, サイズ）
 
    # ファイルからフレームを1枚ずつ取得して動画処理後に保存する
    while True:
        ret, frame = movie.read()        # フレームを取得
        video.write(frame)               # 動画を保存する
        # フレームが取得できない場合はループを抜ける
        if not ret:
            break
    # 撮影用オブジェクトとウィンドウの解放
    movie.release()
    return

In [ ]:
path_in = 'mv/afleg.mp4'          # 元動画のパス
path_out = 'mv/afleg_render.mp4'      # 保存する動画のパス
scale_factor = 10              # FPSにかけるスケールファクター
color_flag = True               # カラー動画はTrue, グレースケール動画はFalse
 
# 動画の再生速度を変更する関数を実行
m_speed_change(path_in, path_out, scale_factor, color_flag)

In [ ]:
saved_mortion = [None, None, None]
start = -100
score = [0, 0, 0]
saved_no = 0

In [ ]:
while True:
  success, img = vid.read()
  imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  results = poses.process(imgRGB)

  if results.pose_landmarks:
         for id,landmark in enumerate(results.pose_landmarks.landmark):
              height, width, channel = img.shape #get dimensions(h height, w width) and the c channel of image
              cx = int(landmark.x * width)
              cy = int(landmark.y * height)
              # cv2.circle(img, (cx, cy), 2, (100, 255, 0), cv2.FILLED)0
              cv2.putText(img, str(id), (cx+10, cy+10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1, cv2.LINE_AA)
              mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS) 

            # ポーズの保存

              if cv2.waitKey(1) & 0xFF == ord('s'):
                saved_mortion[0] = landmark2np(results.pose_landmarks)
                start = time.time()
                saved_no = 1
                print('no.1 saved')
            
              if cv2.waitKey(1) & 0xFF == ord('d'):
                saved_mortion[1] = landmark2np(results.pose_landmarks)
                start = time.time()
                saved_no = 2
                print('no.2 saved')
            
              if cv2.waitKey(1) & 0xFF == ord('f'):
                saved_mortion[2] = landmark2np(results.pose_landmarks)
                start = time.time()
                saved_no = 3
                print('no.3 saved')

              # cos類似度でチェック
              if saved_mortion[0] is not None:
                  now_array = landmark2np(results.pose_landmarks)
                  score[0] = manual_cos(saved_mortion[0], now_array)

              if saved_mortion[1] is not None:
                  now_array = landmark2np(results.pose_landmarks)
                  score[1] = manual_cos(saved_mortion[1], now_array)

              if saved_mortion[2] is not None:
                  now_array = landmark2np(results.pose_landmarks)
                  score[2] = manual_cos(saved_mortion[2], now_array)
  # 3s 表示
  if time.time() - start < 3:
      cv2.putText(img, f'No.{saved_no} saved', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 3.0, (255, 255, 255), thickness=2)

  elif score[0] > 0.9:
      cv2.putText(img, 'no.1 pose', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 3.0, (255, 0, 255), thickness=2)

  elif score[1] > 0.9:
      cv2.putText(img, 'no.2 pose', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 3.0, (255, 0, 255), thickness=2)

  elif score[2] > 0.9:
      cv2.putText(img, 'no.3 pose', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 3.0, (255, 0, 255), thickness=2)
  cv2.imshow("Image", img)

  if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()